In [1]:
import pandas as pd

In [2]:
!pip install scrapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 271 kB 4.7 MB/s 
     |████████████████████████████████| 4.0 MB 66.7 MB/s 
     |████████████████████████████████| 93 kB 3.6 MB/s 
     |████████████████████████████████| 57 kB 6.5 MB/s 
     |████████████████████████████████| 261 kB 70.7 MB/s 
     |████████████████████████████████| 3.1 MB 50.4 MB/s 
     |████████████████████████████████| 74 kB 4.4 MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.6-py3-none-any.whl size=11959 sha256=9172a676ff5a77b1c768e31dd05b26e4268b89bb4c6b840f89198ed601507942
  Stored in directory: /root/.cache/pip/wheels/dc/b9/4a/948b1176e084b9e3f85e4ffc3d08f817b1fdf0d973bbb94f81
Successfully built PyDispatcher


In [3]:
!scrapy startproject amazonscrapper

New Scrapy project 'amazonscrapper', using template directory '/usr/local/lib/python3.8/dist-packages/scrapy/templates/project', created in:
    /content/amazonscrapper

You can start your first spider with:
    cd amazonscrapper
    scrapy genspider example example.com


In [4]:
!scrapy genspider amazon_review haha.com

Created spider 'amazon_review' using template 'basic' 


In [5]:
%cd amazonscrapper

/content/amazonscrapper


In [6]:
!scrapy genspider amazon_review haha.com

Created spider 'amazon_review' using template 'basic' in module:
  amazonscrapper.spiders.amazon_review


In [7]:
!scrapy list

amazon_review


In [8]:
%%writefile amazonscrapper/spiders/amazon_review.py

import scrapy

review_url = 'https://www.amazon.com/product-reviews/{}'

product_list_woman_1 = ['B0169P8Y1Q', 'B078RK1RJ2', 'B0823H7DXH', 'B079ZZC81K', 'B081GCF2QC']
product_list_woman_2 = ['B071K9P1MZ', 'B0711CSPNG', 'B07L4DT2HZ', 'B074SZX5K8', 'B016YKKHOI', 'B00SCD250K', 'B07QM6YH1G', 'B07DX77ZV7', 'B016YKL66G', 'B077R7LV2K']

class AmazonReviewSpider(scrapy.Spider):
    name = 'amazon_review'
    user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'
    def start_requests(self):
        for product in product_list_woman_2:
            url = review_url.format(product)
            yield scrapy.Request(url)

    def parse(self, response):
        for review in response.css('[data-hook="review"]'):
            item = {
                'stars': review.css('[data-hook="review-star-rating"] span ::text').get(),
                'title': review.css('[data-hook="review-title"] span ::text').get(),
                'body': review.xpath('normalize-space(.//*[@data-hook="review-body"])').get()
            }
            
            yield item
        
        next_page = response.xpath('//a[text()="Next page"]/@href').get()
        
        if next_page:
            yield scrapy.Request(response.urljoin(next_page))

Overwriting amazonscrapper/spiders/amazon_review.py


In [9]:
!scrapy crawl amazon_review -o review_10_product_women.csv

Streaming output truncated to the last 5000 lines.
{'stars': '5.0 out of 5 stars', 'title': 'Jeans keep there shape and are super comfortable!', 'body': 'Love Love Love these jeans! I wish the other washes weren’t sold out in my size! I’m bummed! I would have bought them all!'}
2022-12-05 18:13:14 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.amazon.com/Signature-Levi-Strauss-Gold-Label/product-reviews/B077R7LV2K/ref=cm_cr_arp_d_paging_btm_476?ie=UTF8&pageNumber=476>
{'stars': '5.0 out of 5 stars', 'title': 'Great jean at great price', 'body': 'Comfortable with stretch. Nice looking dark jean. Price is excellent.'}
2022-12-05 18:13:14 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.amazon.com/GRECERELLE-Womens-Casual-Pocket-Sleeve/product-reviews/B0711CSPNG/ref=cm_cr_arp_d_paging_btm_466?ie=UTF8&pageNumber=466>
{'stars': '4.0 out of 5 stars', 'title': 'Love the color. Material is nice.', 'body': 'Wear around home'}
2022-12-05 18:13:14 [scrapy.core.scraper] DEBUG

In [10]:
data = pd.read_csv('/content/amazonscrapper/review_10_product_women.csv')

In [11]:
data.head()

stars                                     title  \
0  4.0 out of 5 stars                               Cute Dress!   
1  4.0 out of 5 stars                                Nice dress   
2  4.0 out of 5 stars  Ended up returning but it’s still nice!!   
3  4.0 out of 5 stars             Cute, comfortable & versatile   
4  4.0 out of 5 stars                A lot bigger than expected   

                                                body  
0  Ordered both size S and M. I’m 5’ and between ...  
1  I ordered this dress for my engagement photos ...  
2  I ended up going with different dress but it’s...  
3  Love the color for Fall! I ordered a M in the ...  
4  Got this dress for Thanksgiving. The color is ...

In [12]:
data.shape

(49802, 3)

In [13]:
data['star'] = data['stars'].apply(lambda x: str(x).split()[0]).astype(float)

In [14]:
data.head()

stars                                     title  \
0  4.0 out of 5 stars                               Cute Dress!   
1  4.0 out of 5 stars                                Nice dress   
2  4.0 out of 5 stars  Ended up returning but it’s still nice!!   
3  4.0 out of 5 stars             Cute, comfortable & versatile   
4  4.0 out of 5 stars                A lot bigger than expected   

                                                body  star  
0  Ordered both size S and M. I’m 5’ and between ...   4.0  
1  I ordered this dress for my engagement photos ...   4.0  
2  I ended up going with different dress but it’s...   4.0  
3  Love the color for Fall! I ordered a M in the ...   4.0  
4  Got this dress for Thanksgiving. The color is ...   4.0

In [15]:
data['sentiment'] = data['star'].apply(lambda x: '1' if x >= 3 else '0')

In [16]:
data.head()

stars                                     title  \
0  4.0 out of 5 stars                               Cute Dress!   
1  4.0 out of 5 stars                                Nice dress   
2  4.0 out of 5 stars  Ended up returning but it’s still nice!!   
3  4.0 out of 5 stars             Cute, comfortable & versatile   
4  4.0 out of 5 stars                A lot bigger than expected   

                                                body  star sentiment  
0  Ordered both size S and M. I’m 5’ and between ...   4.0         1  
1  I ordered this dress for my engagement photos ...   4.0         1  
2  I ended up going with different dress but it’s...   4.0         1  
3  Love the color for Fall! I ordered a M in the ...   4.0         1  
4  Got this dress for Thanksgiving. The color is ...   4.0         1

In [19]:
data['sentiment'].unique()

array(['1', '0'], dtype=object)

In [25]:
data['sentiment'].value_counts()

1    45182
0     4620
Name: sentiment, dtype: int64